In [1]:
# !pip install findspark
# !pip install pyspark

In [2]:
# import findspark
# findspark.init()

In [3]:
# #Initializing PySpark
# from pyspark import SparkContext, SparkConf

# # #Spark Config
# conf = SparkConf().setAppName("sample_app")
# sc = SparkContext(conf=conf)

In [4]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import col,sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

In [5]:
%time
from pyspark.sql import SparkSession
#sc.stop()
spark = SparkSession.builder\
        .master("local[2]")\
        .appName("8INF919D1")\
        .getOrCreate()
       

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
22/10/12 11:11:42 WARN Utils: Your hostname, Mac-Pro-de-MARCELO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.62 instead (on interface en2)
22/10/12 11:11:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/12 11:11:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
%time
from pyspark import SparkContext
sc = spark.sparkContext.getOrCreate()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [7]:
%time
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

# # Load and parse the data file into an RDD of LabeledPoint.
# data = MLUtils.loadLibSVMFile(sc, 'iris_libsvm.txt')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", StringType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", StringType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", StringType(), True),
    StructField("capital_loss", StringType(), True),
    StructField("hours_per_week", StringType(), True),
    StructField("native_country", StringType(), True),
    StructField("class", StringType(), True),])

file = '/Users/marcelovasconcellos/PycharmProjects/8INF919_Devoir1_Classification-distribuée-par-arbre-de-décision/dataset/adult.data'
df = spark.read.csv(
    file, 
    header=False, 
    schema=schema
)
df.show(5)

+---+-----------------+-------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country| class|
+---+-----------------+-------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov|  77516| Bachelors|           13|      Never-married|      Adm-clerical| Not-in-family| White|   Male|        2174|           0|            40| United-States| <=50K|
| 50| Self-emp-not-inc|  83311| Bachelors|           13| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|           0|           0|            13| United-States| <=50K|
| 38|          Private| 215646|   HS-grad|            9

In [9]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: string (nullable = true)
 |-- capital_loss: string (nullable = true)
 |-- hours_per_week: string (nullable = true)
 |-- native_country: string (nullable = true)
 |-- class: string (nullable = true)



In [10]:
df.createOrReplaceTempView("CastExample")
df = spark.sql("""
SELECT  CASE
        WHEN TRIM(class) = '>50K' THEN 1
        ELSE 0 END AS label,
        INT(age) AS age, 
        TRIM(workclass) AS workclass,
        INT(fnlwgt) AS fnlwgt,
        TRIM(education) AS education,
        INT(education_num) AS education_num,
        TRIM(marital_status) AS marital_status,
        TRIM(occupation) AS occupation,
        TRIM(relationship) AS relationship,
        TRIM(race) AS race,
        TRIM(sex) AS sex,
        INT(capital_gain) AS capital_gain,
        INT(capital_loss) AS capital_loss,
        INT(hours_per_week) AS hours_per_week,
        TRIM(native_country) AS native_country
FROM CastExample""")

In [11]:
df.show(5)

+-----+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+
|label|age|       workclass|fnlwgt|education|education_num|    marital_status|       occupation| relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|
+-----+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+
|    0| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States|
|    0| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States|
|    0| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-

In [12]:
df.printSchema()

root
 |-- label: integer (nullable = false)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)



In [13]:
spark.sql("SELECT DISTINCT class, COUNT(*) FROM CastExample GROUP BY class").show()

+------+--------+
| class|count(1)|
+------+--------+
|  >50K|    7841|
| <=50K|   24720|
+------+--------+



In [14]:
%time
df.first()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


Row(label=0, age=39, workclass='State-gov', fnlwgt=77516, education='Bachelors', education_num=13, marital_status='Never-married', occupation='Adm-clerical', relationship='Not-in-family', race='White', sex='Male', capital_gain=2174, capital_loss=0, hours_per_week=40, native_country='United-States')

In [15]:
def one_hot_encode(df, column_name):
    distinct_values = df.select(column_name)\
                        .distinct()\
                        .rdd\
                        .flatMap(lambda x: x).collect()

    for distinct_value in distinct_values:
        function = udf(lambda item: 
                       1 if item == distinct_value else 0, 
                       IntegerType())
        new_column_name = column_name+'_'+distinct_value.replace('-', '_').replace('?', 'unknown').lower()
        df = df.withColumn(new_column_name, function(col(column_name)))
    return df.drop(column_name)

In [16]:
columns = [
    'workclass', 
    'education', 
    'marital_status', 
    'occupation', 
    'relationship', 
    'race', 
    'sex', 
    'native_country', ]
for column in columns:
    df = one_hot_encode(df, column)

In [17]:
df.show()

22/10/12 11:11:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+---+------+-------------+------------+------------+--------------+--------------------------+-------------------+-------------------+-----------------+---------------------+---------------------+----------------------+-----------------+----------------------+-----------------+--------------+-----------------+--------------------+-------------------+-----------------+-------------+-----------------+-------------------+--------------+-----------------+-------------------+--------------+-------------------+----------------------+---------------------+------------------------+----------------------------+------------------------------------+-----------------------+----------------------+--------------------------------+---------------------------------+----------------+--------------------------+-------------------------+----------------------------+--------------------------+-----------------------+---------------------------+--------------------------+--------------------------+--

In [18]:
temp = df.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))
temp.take(5)

22/10/12 11:12:05 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 31 (TID 22): Attempting to kill Python Worker


[LabeledPoint(0.0, [39.0,77516.0,13.0,2174.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [50.0,83311.0,13.0,0.0,0.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [38.0,215646.0,9.0,0.0,0.0,40.0,0.0,0.0,0.0,1.0,0.0,0.

In [19]:
%time
(trainingData, testData) = temp.randomSplit([0.7, 0.3])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


In [20]:
%time
model = DecisionTree.trainClassifier(
    trainingData, 
    numClasses=2, 
    categoricalFeaturesInfo={},
    impurity='entropy', 
    maxDepth=5, 
    maxBins=32)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


22/10/12 11:12:09 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 32 (TID 23): Attempting to kill Python Worker


In [21]:
%time
print(model.toDebugString())

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 9.06 µs
DecisionTreeModel classifier of depth 5 with 27 nodes
  If (feature 37 <= 0.5)
   If (feature 3 <= 7833.0)
    Predict: 0.0
   Else (feature 3 > 7833.0)
    If (feature 0 <= 19.5)
     Predict: 0.0
    Else (feature 0 > 19.5)
     If (feature 0 <= 66.5)
      Predict: 1.0
     Else (feature 0 > 66.5)
      If (feature 29 <= 0.5)
       Predict: 1.0
      Else (feature 29 > 0.5)
       Predict: 0.0
  Else (feature 37 > 0.5)
   If (feature 2 <= 12.5)
    If (feature 3 <= 3414.5)
     Predict: 0.0
    Else (feature 3 > 3414.5)
     If (feature 3 <= 7833.0)
      If (feature 5 <= 29.5)
       Predict: 0.0
      Else (feature 5 > 29.5)
       Predict: 1.0
     Else (feature 3 > 7833.0)
      If (feature 2 <= 4.5)
       Predict: 0.0
      Else (feature 2 > 4.5)
       Predict: 1.0
   Else (feature 2 > 12.5)
    If (feature 3 <= 3414.5)
     If (feature 4 <= 1695.5)
      If (feature 5 <= 30.5)
       Predict: 0.0
      Else (fe

In [22]:
%time
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


Test Error = 0.16594157862026104
Learned classification tree model:


In [23]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [24]:
# Instantiate metrics object
metrics = BinaryClassificationMetrics(labelsAndPredictions)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Area under PR = 0.433911372388986
Area under ROC = 0.7750028162667568
